In [14]:
import os
os.chdir('/home/ubuntu/code/pump_post_midterm/pump')

In [15]:
# # Get test_q_keys
# import random
# import pandas as pd
# from src.utils import get_file_from_s3

# survey_name = "American_Trends_Panel_W34"
# random.seed(42)
# survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
# q_keys = list(survey_df['key'])
# test_q_keys = random.choices(q_keys, k=5)

# print(test_q_keys)

In [16]:
import random
random.seed(42)

from src.utils import TEST_KEY_MAPPING

test_q_keys_mapping = TEST_KEY_MAPPING

# test_q_keys_mapping = {
#     "American_Trends_Panel_W26": ['GUNRESPNOKIDSB_W26', 'WORLDDANGER_W26', 'GUNIDENTITY_W26', 'REASONGUNC_W26', 'GUNRESPKIDSC_W26'],
#     "American_Trends_Panel_W27": ['CAREGIV3A_W27', 'PREDICTA_W27', 'WORK3C_W27', 'CARS10D_W27', 'CAREGIV7_W27'],
#     "American_Trends_Panel_W29": ['TRAITSD_W29', 'NOWSMK_NHIS_W29', 'HELPHURTE_W29', 'HELPHURTA_W29', 'GIRLSF2A_W29'],
#     "American_Trends_Panel_W32": ['NEIGHINTERB_W32', 'SATLIFEC_W32', 'SUCCESSIMPA_W32', 'COMMIMPE_W32', 'GROWUPUSR_W32'],
#     "American_Trends_Panel_W34": ['MED4A_W34', 'SCI2A_W34', 'FUD22_W34', 'EAT3H_W34', 'MED6D_W34']
# }


# Hyper parameters

In [17]:

survey_names = [
    "American_Trends_Panel_W26",
    "American_Trends_Panel_W27",
    "American_Trends_Panel_W29",
    "American_Trends_Panel_W32",
    "American_Trends_Panel_W34"
]

def skip_exp(filename):
    # # check sonnet vs haiku pred
    # if 'v11' in filename: return True
    # if ('0830' in filename and 'prompt2' in filename): return False
    # if ('sonnetpred' in filename): return False
    # return True

    # # check prompts
    # if 'haikupred' not in filename: return True
    # # if 'v12' in filename and 'prompt2' in filename: return False
    # if 'v11' in filename: 
    #     if 'prompt2' in filename: return False
    #     if 'prompt3' in filename: return False
    #     if 'prompt4' in filename: return False
    # if 'v10' in filename and 'prompt2' in filename: return False
    # return True

    # # check runs
    # if 'haikupred' not in filename: return True
    # if '0831' in filename and 'prompt3' in filename: return False
    # if 'v11' in filename:
    #     if 'prompt3' in filename: return False
    # if 'v10' in filename and 'prompt2' in filename: return False
    # return True

    # check known test
    if 'v13' not in filename: return True


def get_exp(filename):
    if filename[-5:] == '.json': filename = filename[:-5]
    # segs = ['history', 'demo', 'persona', 'v11', 'v10', 'prompt3', 'prompt4', 'run1', 'run2', 'run3']
    segs = ['history', 'demo', 'persona', 'v13', 'prompt3']
    exp_elements = [_ for _ in filename.split('_') if _ in segs]
    final = '_'.join(exp_elements)
    # print(filename, '->', final)
    return final

# Main

In [18]:
import json
import os
import pandas as pd
import numpy as np
import pandas as pd
import random

from collections import Counter
from src.utils import get_file_from_s3

def get_full_results(survey_name):
    res = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue
        exp = get_exp(filename)
        # exp = filename
        if exp not in res:
            res[exp] = []
            
        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        res[exp] += [f"={sum([_['is_correct'] for _ in data])}/{len(data)}", sum([_['is_correct'] for _ in data])/len(data)]


    df = pd.DataFrame(res)
    df = df[sorted(df.columns)]
    df = df.transpose()

    acc_df = df
    acc_df = acc_df.rename(columns={0: "acc_cnt", 1: "acc"})
    acc_df.sort_values(by='acc', ascending=False)


    test_q_keys = test_q_keys_mapping[survey_name]
    print(test_q_keys)
    survey_df = pd.read_csv(get_file_from_s3(f"human_resp/{survey_name}/info.csv"))
    sub_df = survey_df[survey_df['key'].isin(test_q_keys)]

    wd_question_mappings = {row['key']: dict(zip(eval(row['references'].lower()), eval(row['option_ordinal']))) for _, row in sub_df.iterrows()}
    wd_question_mappings

    records = {}
    for filename in os.listdir(f'opinions_qa/output/{survey_name}/'):
        if skip_exp(filename): continue

        with open(f"opinions_qa/output/{survey_name}/{filename}", 'r') as f:
            data = json.load(f)
        
        exp = get_exp(filename)

        res = []
        pred_missing_cnt = 0
        gold_missing_cnt = 0
        for entry in data:
            q_idx = entry['q_idx']
            q_key = test_q_keys[q_idx]
            pos_mapping = wd_question_mappings[q_key]
            prediction = entry['prediction'].lower()
            gold_answer = entry['gold_answer'].lower()
            if gold_answer not in pos_mapping:
                gold_missing_cnt += 1
                continue
            if prediction not in pos_mapping:
                pred_missing_cnt += 1
                if prediction not in entry['references']:
                    refs = '/'.join(entry['references'].split('\n'))
                    print(f"Prediction: |{prediction}|, Gold: |{gold_answer}|, References: |{refs}|")
                continue
                prediction = random.choice(list(pos_mapping.keys()))
            pred_pos = pos_mapping[prediction]
            gold_pos = pos_mapping[gold_answer]
            res.append(abs(pred_pos-gold_pos))
        records[exp] = [Counter(res), np.mean(res), pred_missing_cnt, gold_missing_cnt]

    q_wd_df = pd.DataFrame(records)
    q_wd_df = q_wd_df[sorted(q_wd_df.columns)].transpose()
    q_wd_df = q_wd_df.rename(columns={0: "q_1wd_cnt", 1: "q_1wd", 2: "q_1wd_pmc", 3: "q_1wd_gmc"})
    q_wd_df

    merged_df = pd.merge(acc_df, q_wd_df, left_index=True, right_index=True)
    # merged_df = pd.merge(merged_df, s_wd_df, left_index=True, right_index=True)
    # merged_df = merged_df[['wd_avg', 'wd_std', 'acc_avg', 'acc_std']]

    merged_df = merged_df.sort_values(by='q_1wd', ascending=True)
    merged_df = merged_df.sort_values(by='acc', ascending=False)

    # merged_df.sort_values(by='acc_avg', ascending=False)
    # merged_df.sort_values(by='wd_avg', ascending=True)
    # merged_df = merged_df.sort_values(by='s_1wd', ascending=True)

    merged_df['acc_cnt'] = merged_df['acc_cnt'].apply(lambda x: '\''+str(x))
    merged_df

    return merged_df

# Results

In [19]:
from IPython.display import display

for survey_name in survey_names:
    print(survey_name)
    merged = get_full_results(survey_name)
    display(merged)
    print('\n\n\n')

American_Trends_Panel_W26
['WORLDDANGER_W26', 'WORRYE_W26', 'GUN_W26', 'GUNIDENTITY_W26', 'GUNSOCIETY_W26', 'GUNCOMMUNITY_W26', 'GUNRESPKIDSC_W26']


,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v13_prompt3_history,'=1525/2207,0.690983,"{0.5: 82, 0.0: 1522, 1.0: 536, 2.0: 36, 3.0: 6}",0.305683,0,25
v13_prompt3_history_demo,'=1499/2207,0.679203,"{0.5: 82, 0.0: 1499, 1.0: 558, 2.0: 34, 3.0: 9}",0.318057,0,25
v13_prompt3_demo,'=1369/2207,0.620299,"{0.5: 82, 0.0: 1369, 1.0: 682, 2.0: 44, 3.0: 5}",0.378552,0,25
v13_prompt3_history_persona,'=1273/2207,0.576801,"{0.5: 78, 0.0: 1273, 1.0: 782, 2.0: 35, 3.0: 14}",0.427589,0,25
v13_prompt3_history_demo_persona,'=1270/2207,0.575442,"{0.5: 78, 0.0: 1270, 1.0: 790, 2.0: 31, 3.0: 13}",0.426214,0,25
v13_prompt3_demo_persona,'=920/1746,0.526919,"{0.5: 60, 0.0: 920, 1.0: 707, 2.0: 29, 3.0: 12}",0.480903,0,18
v13_prompt3_persona,'=1156/2207,0.523788,"{0.5: 78, 0.0: 1156, 1.0: 901, 2.0: 32, 3.0: 15}",0.480752,0,25






American_Trends_Panel_W27
['WORK4A_W27', 'WORK4B_W27', 'WORK4C_W27', 'ROBJOB2_W27', 'ROBJOB3A_W27', 'ROBJOB3B_W27', 'ROBJOB4C_W27', 'ROBJOB5B_W27', 'ROBJOB5D_W27']
Prediction: |positive impact|, Gold: |more interesting|, References: |More interesting/Less interesting/No impact either way/Refused|
Prediction: |positive impact|, Gold: |no impact either way|, References: |More interesting/Less interesting/No impact either way/Refused|
Prediction: |positive impact|, Gold: |no impact either way|, References: |More interesting/Less interesting/No impact either way/Refused|
Prediction: |positive impact|, Gold: |no impact either way|, References: |More interesting/Less interesting/No impact either way/Refused|
Prediction: |positive impact|, Gold: |more interesting|, References: |More interesting/Less interesting/No impact either way/Refused|
Prediction: |positive impact|, Gold: |no impact either way|, References: |More interesting/Less interesting/No impact either way/Refused|
Prediction: 

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v13_prompt3_history,'=1648/3255,0.506298,"{1.0: 1182, 0.0: 1648, 2.0: 95, 0.5: 265, 3.0: 7}",0.477166,33,25
v13_prompt3_history_demo_persona,'=1637/3255,0.502919,"{2.0: 86, 0.0: 1637, 1.0: 1181, 0.5: 314, 3.0:...",0.478638,0,25
v13_prompt3_history_persona,'=1585/3255,0.486943,"{1.0: 1224, 0.0: 1585, 0.5: 283, 2.0: 125, 3.0...",0.512229,0,25
v13_prompt3_demo,'=1413/3075,0.459512,"{1.0: 1255, 0.0: 1413, 2.0: 141, 0.5: 239, 3.0...",0.548297,0,21






American_Trends_Panel_W29
['MASC2AF1_W29', 'SEENMASC_W29', 'SEENFEM_W29', 'MAN1E_W29', 'BOYSF1A_W29', 'BOYSF1B_W29', 'GIRLSF2A_W29']
Prediction: |very womanly or feminine|, Gold: |not too important|, References: |Very important/Somewhat important/Not too important/Not at all important/Refused|


,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v13_prompt3_history,'=894/1566,0.570881,"{1.0: 404, 0.5: 171, 0.0: 888, 2.0: 73, 3.0: 12}",0.433786,0,18
v13_prompt3_history_demo,'=888/1566,0.56705,"{1.0: 399, 0.5: 181, 0.0: 888, 2.0: 70, 3.0: 10}",0.426034,0,18
v13_prompt3_history_persona,'=881/1566,0.56258,"{1.0: 422, 0.5: 175, 0.0: 881, 2.0: 62, 3.0: 7}",0.423077,1,18
v13_prompt3_history_demo_persona,'=853/1566,0.5447,"{1.0: 411, 0.5: 188, 0.0: 851, 2.0: 78, 3.0: 20}",0.465762,0,18
v13_prompt3_demo_persona,'=746/1566,0.476373,"{1.0: 461, 0.5: 258, 0.0: 746, 2.0: 76, 3.0: 7}",0.492894,0,18
v13_prompt3_persona,'=729/1566,0.465517,"{0.0: 729, 0.5: 271, 1.0: 456, 2.0: 82, 3.0: 10}",0.507429,0,18
v13_prompt3_demo,'=724/1566,0.462324,"{2.0: 78, 0.0: 724, 1.0: 437, 0.5: 301, 3.0: 8}",0.495801,0,18






American_Trends_Panel_W32
['SOCTRUST_W32', 'COMATTACH_W32', 'JOBSFUTURE_W32', 'NEIGHKEYS_W32', 'IMMIMPACT_W32', 'ETHNCMAJ_W32', 'WHADVANT_W32', 'GAYMARR2_W32', 'ABORTION_W32']
Prediction: |refused|, Gold: |yes|, References: |Yes/No/Refused|
Prediction: |refused|, Gold: |neither positive nor negative|, References: |Mostly positive/Mostly negative/Neither positive nor negative/Refused|
Prediction: |refused|, Gold: |neither positive nor negative|, References: |Mostly positive/Mostly negative/Neither positive nor negative/Refused|
Prediction: |refused|, Gold: |legal in all or most cases|, References: |legal in all or most cases/illegal in all or most cases/Refused|
Prediction: |refused|, Gold: |yes|, References: |Yes/No/Refused|
Prediction: |refused|, Gold: |illegal in all or most cases|, References: |legal in all or most cases/illegal in all or most cases/Refused|
Prediction: |only a few|, Gold: |mostly positive|, References: |Mostly positive/Mostly negative/Neither positive nor negat

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v13_prompt3_history,'=2263/5002,0.452419,"{1.0: 1564, 0.5: 876, 0.0: 2263, 2.0: 220, 3.0...",0.50617,12,47
v13_prompt3_history_demo_persona,'=2142/5002,0.428229,"{0.0: 2142, 0.5: 859, 1.0: 1679, 3.0: 11, 2.0:...",0.537568,4,47
v13_prompt3_history_persona,'=609/1556,0.391388,"{0.0: 609, 0.5: 273, 1.0: 552, 2.0: 94, 3.0: 4}",0.579961,2,22






American_Trends_Panel_W34
['SCI1_W34', 'EAT1_W34', 'FUD35_W34', 'MED2G_W34', 'MED6D_W34', 'MED6E_W34']
Prediction: |most of it|, Gold: |all of the time|, References: |All of the time/More than half of the time/About half of the time/Less than half of the time/Never/Refused|
Prediction: |most of it|, Gold: |all of the time|, References: |All of the time/More than half of the time/About half of the time/Less than half of the time/Never/Refused|
Prediction: |most of it|, Gold: |about half of the time|, References: |All of the time/More than half of the time/About half of the time/Less than half of the time/Never/Refused|
Prediction: |most of it|, Gold: |about half of the time|, References: |All of the time/More than half of the time/About half of the time/Less than half of the time/Never/Refused|
Prediction: |most of it|, Gold: |all of the time|, References: |All of the time/More than half of the time/About half of the time/Less than half of the time/Never/Refused|
Prediction: |most o

,acc_cnt,acc,q_1wd_cnt,q_1wd,q_1wd_pmc,q_1wd_gmc
v13_prompt3_history_demo,'=910/1518,0.599473,"{0.0: 910, 2.0: 99, 1.0: 483, 3.0: 15}",0.481752,3,8
v13_prompt3_history,'=906/1518,0.596838,"{0.0: 906, 1.0: 461, 2.0: 123, 3.0: 19}",0.506296,1,8
v13_prompt3_history_demo_persona,'=843/1518,0.555336,"{1.0: 563, 0.0: 843, 2.0: 92, 3.0: 12}",0.518543,0,8
v13_prompt3_history_persona,'=772/1518,0.508564,"{1.0: 622, 0.0: 772, 2.0: 97, 3.0: 15}",0.571713,4,8
v13_prompt3_demo,'=708/1518,0.466403,"{0.0: 708, 1.0: 595, 2.0: 175, 3.0: 31, 4.0: 1}",0.690066,0,8
v13_prompt3_demo_persona,'=663/1518,0.436759,"{1.0: 672, 0.0: 663, 2.0: 162, 3.0: 13}",0.68543,0,8
v13_prompt3_persona,'=626/1518,0.412385,"{1.0: 713, 0.0: 626, 2.0: 159, 3.0: 12}",0.706623,0,8
